## Notebook Setup

### Imports

In [1]:
from ikt457.src.common_imports import *
from ikt457.src.config import get_paths
import random
import matplotlib.animation as animation
from IPython.display import display, HTML

### Classes

#### Environment

In [2]:
class Environment:
    def __init__(self, c_1, c_2):
        self.c_1 = c_1
        self.c_2 = c_2
        
    def reward_probability(self, M):
        if M <= 3:
            return M * self.c_1
        else:
            return self.c_2 - (M - 3) * self.c_1

#### Tsetlin

In [3]:
class Tsetlin:
    def __init__(self, n):
        self.n = n
        self.state = random.choice([self.n, self.n+1])

    def reward(self):
        if self.state <= self.n and self.state > 1:
            self.state -= 1
        elif self.state > self.n and self.state < 2*self.n:
            self.state += 1

    def penalize(self):
        if self.state <= self.n:
            self.state += 1
        elif self.state > self.n:
            self.state -= 1

    def makeDecision(self):
        return "Yes" if self.state <= self.n else "No"

### Functions

#### Save and link plot

In [4]:
def save_and_link_plot(plt, filename):
    # Save the plot
    plt.savefig(filename)
    plt.clf()  # Clear the current figure to avoid showing it in the notebook
    
    # Create a clickable link to the plot
    link = f'<a href="{filename}" target="_blank">Click here to view the plot: {filename}</a>'
    display(HTML(link))

#### Save and link gif

In [5]:
def save_and_link_gif(animation, filename, writer='pillow', fps=10):
    # Save the GIF
    animation.save(filename, writer=writer, fps=fps)
    
    # Close the figure to prevent it from displaying in the notebook
    plt.close(animation._fig)
    
    # Create a clickable link to the GIF
    link = f'<a href="{filename}" target="_blank">Click here to view the GIF: {filename}</a>'
    display(HTML(link))

### Global Variables

#### Paths

In [6]:
paths = get_paths()

#### Other

In [7]:
c_1 = 0.2
c_2 = 0.6
n_states = 3
n_ta = 5
n_iterations = 300

VERBOSE = False

## Task

In [8]:
# Step 0: Initialize the environment
env = Environment(c_1, c_2)

# Step 1: Create 5 Tsetlin Automata
automata = [Tsetlin(n_states) for _ in range(n_ta)]

# Track the states over time
states_over_time = []

# Track the number of "Yes" actions for each automaton over time
yes_counts_over_time = []

# Track the number of "Yes" actions for each automaton
yes_counts = [0] * n_ta

for i in range(n_iterations):
    # Count the number of "Yes" actions
    for index, automaton in enumerate(automata):
        if automaton.makeDecision() == "Yes":
            yes_counts[index] += 1
    
    # Store the current yes_counts for this iteration
    yes_counts_over_time.append(yes_counts.copy())
    
    # Step 2: Count the number of "Yes" actions
    yes_count = sum(1 for automaton in automata if automaton.makeDecision() == "Yes")
    
    # Print out the states or actions to observe the process
    actions = [automaton.makeDecision() for automaton in automata]
    states = [automaton.state for automaton in automata]
    if VERBOSE:
        print(f"{actions} | {yes_count} Yes | {states}")
        
    # Step 3: Reward / Penalty
    reward_prob = env.reward_probability(yes_count)
    
    # Loop through the automata and reward or penalize them
    for automaton in automata:
        if random.random() <= reward_prob:
            automaton.reward()
        else:
            automaton.penalize()
    
    # Record states for each automaton
    for index, automaton in enumerate(automata):
        states_over_time.append({
            'Iteration': i,
            'Automaton': index,
            'State': automaton.state
        })

## Visualization

### Static plots

In [9]:
plt.figure(figsize=(10, 6))
automaton_labels = [f'Automaton {i+1}' for i in range(n_ta)]
plt.bar(automaton_labels, yes_counts, color='skyblue')
plt.xlabel('Automaton')
plt.ylabel('Number of "Yes" Actions')
plt.title('Number of "Yes" Actions per Automaton')

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/yes_actions_per_automaton_{n_ta}_{n_iterations}.png")

<Figure size 1000x600 with 0 Axes>

In [10]:
plt.figure(figsize=(12, 6))
for automaton_index in range(n_ta):
    automaton_states = [state['State'] for state in states_over_time if state['Automaton'] == automaton_index]
    plt.plot(range(len(automaton_states)), automaton_states, label=f'Automaton {automaton_index}')
plt.xlabel('Iteration')
plt.ylabel('State')
plt.title('State of Each Tsetlin Automaton Over Time')
plt.legend()

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_over_time_{n_ta}_{n_iterations}.png")


<Figure size 1200x600 with 0 Axes>

In [11]:
# Create a matrix of states
state_matrix = np.zeros((n_iterations, n_ta))

for state in states_over_time:
    iteration = state['Iteration']
    automaton = state['Automaton']
    state_matrix[iteration, automaton] = state['State']

plt.figure(figsize=(12, 6))
sns.heatmap(state_matrix, cmap='Blues', cbar_kws={'label': 'State'})
plt.xlabel('Automaton')
plt.ylabel('Iteration')
plt.title('Heatmap of Automaton States Over Time')

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_heatmap_{n_ta}_{n_iterations}.png")


<Figure size 1200x600 with 0 Axes>

In [12]:
# Convert to DataFrame for plotting
df_states = pd.DataFrame(states_over_time)

plt.figure(figsize=(12, 6))
sns.boxplot(x='Iteration', y='State', data=df_states)
plt.xlabel('Iteration')
plt.ylabel('State')
plt.title('Box Plot of States of Tsetlin Automata Over Time')

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_boxplot_{n_ta}_{n_iterations}.png")

<Figure size 1200x600 with 0 Axes>

In [13]:
plt.figure(figsize=(12, 6))
for automaton_index in range(n_ta):
    automaton_states = [state['State'] for state in states_over_time if state['Automaton'] == automaton_index]
    plt.scatter(range(len(automaton_states)), automaton_states, label=f'Automaton {automaton_index}')
plt.xlabel('Iteration')
plt.ylabel('State')
plt.title('Scatter Plot of State Changes for Each Tsetlin Automaton')
plt.legend()

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_scatter_{n_ta}_{n_iterations}.png")


<Figure size 1200x600 with 0 Axes>

### 3D plots

In [14]:
from mpl_toolkits.mplot3d import Axes3D

# Prepare data for 3D line plot
iterations = range(n_iterations)
automaton_indices = range(n_ta)
states_matrix = np.zeros((n_iterations, n_ta))

for i in range(n_iterations):
    for j in range(n_ta):
        states_matrix[i, j] = automata[j].state

# Create a 3D plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting each automaton's state over time
for j in automaton_indices:
    ax.plot(iterations, [j]*n_iterations, states_matrix[:, j], label=f'Automaton {j+1}')

ax.set_xlabel('Iteration')
ax.set_ylabel('Automaton')
ax.set_zlabel('State')
ax.set_title('3D Visualization of Automaton States Over Time')
plt.legend()

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_3d_{n_ta}_{n_iterations}.png")


<Figure size 1200x800 with 0 Axes>

In [15]:
# Create a 3D surface plot
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

X, Y = np.meshgrid(iterations, automaton_indices)
Z = states_matrix.T

ax.plot_surface(X, Y, Z, cmap='viridis')

ax.set_xlabel('Iteration')
ax.set_ylabel('Automaton')
ax.set_zlabel('State')
ax.set_title('3D Surface Plot of Automaton States Over Time')

save_and_link_plot(plt, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/automaton_states_surface_{n_ta}_{n_iterations}.png")


<Figure size 1200x800 with 0 Axes>

### Gif's

In [16]:
# Define the bar plot update function
def update_bar(num, yes_counts_over_time):
    plt.clf()  # Clear the current figure
    automaton_labels = [f'Automaton {i+1}' for i in range(n_ta)]
    plt.bar(automaton_labels, yes_counts_over_time[num], color='skyblue')
    plt.xlabel('Automaton')
    plt.ylabel('Number of "Yes" Actions')
    plt.title(f'Number of "Yes" Actions per Automaton (Iteration {num+1})')
    plt.ylim(0, max(max(yes_counts_over_time))+1)

# Create the figure and bar plot
fig = plt.figure(figsize=(10, 6))

# Create the animation object
bar_animation = animation.FuncAnimation(fig, update_bar, frames=len(yes_counts_over_time), fargs=(yes_counts_over_time,), interval=50)

# Save the animation as a GIF
save_and_link_gif(bar_animation, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/yes_actions_per_automaton_{n_ta}_{n_iterations}.gif")



In [17]:
# Define the line plot update function
def update_line(num, yes_counts_over_time):
    plt.clf()  # Clear the current figure
    for i in range(n_ta):
        automaton_counts = [counts[i] for counts in yes_counts_over_time[:num+1]]
        plt.plot(range(num+1), automaton_counts, label=f'Automaton {i+1}')
    plt.xlabel('Iteration')
    plt.ylabel('Cumulative "Yes" Actions')
    plt.title(f'Cumulative "Yes" Actions Over Time (Iteration {num+1})')
    plt.legend(loc='upper left')
    plt.ylim(0, 1.5 * max(max(yes_counts_over_time))+1)

# Create the figure and plot
fig = plt.figure(figsize=(10, 6))

# Create the animation object
line_animation = animation.FuncAnimation(fig, update_line, frames=len(yes_counts_over_time), fargs=(yes_counts_over_time,), interval=50)

# Save the animation as a GIF
save_and_link_gif(line_animation, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/cumulative_yes_actions_{n_ta}_{n_iterations}.gif")


In [18]:
# Prepare data for the heatmap
state_matrix = np.zeros((n_iterations, n_ta))

for state in states_over_time:
    iteration = state['Iteration']
    automaton = state['Automaton']
    state_matrix[iteration, automaton] = state['State']

# Define the heatmap update function
def update_heatmap(num, state_matrix):
    plt.clf()  # Clear the current figure
    sns.heatmap(state_matrix[:num+1, :], cmap='viridis', cbar=True)
    plt.xlabel('Automaton')
    plt.ylabel('Iteration')
    plt.title(f'State Heatmap of Automata (Iteration {num+1})')

# Create the figure and plot
fig = plt.figure(figsize=(10, 6))

# Create the animation object
heatmap_animation = animation.FuncAnimation(fig, update_heatmap, frames=n_iterations, fargs=(state_matrix,), interval=50)

# Save the animation as a GIF
save_and_link_gif(heatmap_animation, f"{paths['PATH_1_LEARNING_AUTOMATA_GAME']}/results/state_heatmap_{n_ta}_{n_iterations}.gif")
